In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.cluster import KMeans

In [37]:
#Dataset importing
df = pd.read_csv("CLVP.csv")

In [38]:
#Removing irrelevant feature. (The name of the customer doesn't contribute to the change in cusstomer life time value.)
#For example: Two people who share a name like Neha could have variying difference in their life time values
df.drop(columns=['Customer'], inplace=True)

In [39]:
# Remove invalid rows (Income = 0)
df = df[df['Income'] > 0]

In [40]:
# Encoding the emplyment column. "Unemplyed" changed to 0 and the other values change to 1
df['IsEmployed'] = df['EmploymentStatus'].apply(lambda x: 0 if x == 'Unemployed' else 1)

In [41]:
# Ordinal encoding for Education
education_mapping = {
    'High School or Below': 0,
    'College': 1,
    'Bachelor': 2,
    'Master': 3,
    'Doctor': 4
}
df['EducationLevel'] = df['Education'].map(education_mapping)

In [42]:
# KMeans clustering for segmentation
kmeans_features = df[['Income', 'IsEmployed', 'EducationLevel']]
kmeans = KMeans(n_clusters=4, random_state=42)
df['ClusterID'] = kmeans.fit_predict(kmeans_features)

In [43]:
# Define target and features
y = df['CustomerLV']
X = df.drop(columns=['CustomerLV', 'Education', 'EmploymentStatus'])

In [44]:
# Separate columns by type
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [45]:
# Preprocessing pipelines
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

In [46]:
# Complete pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
])

In [47]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Fit and predict
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [49]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")


RMSE: 6753.96
MAE: 4307.36
